# Training

In [5]:
!pip install ultralytics --upgrade --force-reinstall

  Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl.metadata (52 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.9.1-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp312-cp312-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd6

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.2.11 requires idna==3.7, but you have idna 3.11 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.9.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 GB 16.4 MB/s eta 0:02:52
   ---------------------------------------- 0.0/2.8 GB 3.0 MB/s eta 0:15:53
   ---------------------------------------- 0.0/2.8 GB 2.4 MB/s eta 0:19:36
   ---------------------------------------- 0.0/2.8 GB 2.3 MB/s eta 0:20:03
   ---------------------------------------- 0.0/2.8 GB 2.2 MB/s eta 0:21:20
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:03
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:38
   ---------------------------------------- 0.0/2.8 GB 2.0 MB/s eta 0:23:01
   ---------------------------------------- 0.0/2.8 GB 2.1 MB/s eta 0:22:46
   ---------------------------------------- 0.0/2.8 GB 2.0 MB/s eta 0:22:53
   ---------------------------------------- 0.0/2


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Model

In [5]:
import torch
from ultralytics import YOLO

print('suskses import YOLO')
print('GPU available:', torch.cuda.is_available())

model = YOLO('yolov8n.pt')

suskses import YOLO
GPU available: True


In [4]:
import torch
import sys

print(f"1. Python Version: {sys.version.split()[0]}")
print(f"2. PyTorch Version: {torch.__version__}")
print(f"3. CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"4. GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"5. CUDA Version di PyTorch: {torch.version.cuda}")
    
    # Tes Alokasi Memori (Memaksa GPU kerja sedikit)
    try:
        x = torch.ones(1).cuda()
        print("6. Test Tensor ke GPU: BERHASIL ✅")
    except Exception as e:
        print(f"6. Test Tensor ke GPU: GAGAL ❌ ({e})")
else:
    print("❌ GPU TIDAK TERDETEKSI SAMA SEKALI.")
    print("   (Kemungkinan besar PyTorch yang terinstall adalah versi CPU)")

1. Python Version: 3.12.9
2. PyTorch Version: 2.7.1+cu118
3. CUDA Available: True
4. GPU Name: NVIDIA GeForce RTX 5070 Laptop GPU
5. CUDA Version di PyTorch: 11.8
6. Test Tensor ke GPU: BERHASIL ✅


## Cek Pembagian Data

In [1]:
import os

In [3]:
dataset_path = 'C:/resep.in/dataset_normalisasi_fix'

def hitung_file(folder_name):
    # Gabungkan path utama dengan nama folder (train/valid/test)
    path_lengkap = os.path.join(dataset_path, folder_name, 'images')
    
    # Cek apakah folder ada
    if os.path.exists(path_lengkap):
        # Hitung jumlah file gambar (jpg, png, jpeg)
        jumlah = len([f for f in os.listdir(path_lengkap) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        return jumlah
    else:
        return 0

# Hitung masing-masing
n_train = hitung_file('train')
n_val   = hitung_file('valid')
if n_val == 0: n_val = hitung_file('val') 
n_test  = hitung_file('test')

total = n_train + n_val + n_test

print("=== STATISTIK DATASET ANDA ===")
print(f"1. Train Set : {n_train} gambar")
print(f"2. Valid Set : {n_val} gambar")
print(f"3. Test Set  : {n_test} gambar")
print(f"-----------------------------")
print(f"TOTAL DATA   : {total} gambar")

if total > 0:
    print(f"\nPersentase Pembagian (Split Ratio):")
    print(f"Train : {n_train/total*100:.1f}%")
    print(f"Valid : {n_val/total*100:.1f}%")
    print(f"Test  : {n_test/total*100:.1f}%")
else:
    print("❌ Folder tidak ditemukan. Cek lagi path 'dataset_path' di atas.")

=== STATISTIK DATASET ANDA ===
1. Train Set : 34381 gambar
2. Valid Set : 3951 gambar
3. Test Set  : 5905 gambar
-----------------------------
TOTAL DATA   : 44237 gambar

Persentase Pembagian (Split Ratio):
Train : 77.7%
Valid : 8.9%
Test  : 13.3%


## Train Data

In [5]:
result = model.train(
    data='C:/resep.in/dataset_normalisasi_fix/data.yaml',
    epochs=50,
    patience=3,
    imgsz=640,
    batch=16,
    device=0,
    name='training_results',
    optimizer='AdamW',
    lr0=0.001,
)

Ultralytics 8.3.252  Python-3.12.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/resep.in/dataset_normalisasi_fix/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=training_results, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True,

c:\Users\Shallom Otniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3451: DecompressionBombWarning: Image size (153225000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


val: Scanning C:\resep.in\dataset_normalisasi_fix\valid\labels... 3951 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3951/3951 630.8it/s 6.3s0.2s
val: C:\resep.in\dataset_normalisasi_fix\valid\images\food_ingredients_photo-1590848791238-fb3596a2ba56_jpg.rf.690f03bd00fe3e18a393272b0f717f1c.jpg: 1 duplicate labels removed
val: New cache created: C:\resep.in\dataset_normalisasi_fix\valid\labels.cache
WARNING Box and segment counts should be equal, but got len(segments) = 424, len(boxes) = 20783. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to C:\resep.in\runs\detect\training_results\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\resep.in\runs\detect\training_result